# Pair A/B testing
## Ejericicios de A/B Testing

In [34]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

pd.set_option('display.max_columns', None) 

import warnings
warnings.filterwarnings("ignore")

sns.set_style("whitegrid")
 
sns.set_theme(style="dark")

plt.style.use('Solarize_Light2')

In [8]:
df_marketing = pd.read_csv("data/marketing_AB.csv", index_col=0)
df_marketing.sample(5)

,user id,test group,converted,total ads,most ads day,most ads hour
310377,919734,psa,False,4,Sunday,13
408710,1111088,ad,False,16,Wednesday,14
201325,1003495,ad,False,2,Tuesday,10
140598,1314294,ad,False,2,Monday,15
566435,1192354,ad,False,24,Sunday,2


### Ejercicio1:
Supongamos que estamos realizando un experimento para determinar si un nuevo diseño de anuncio (test group "ad") tiene un impacto significativo en la tasa de conversión en comparación con el diseño de anuncio anterior (test group "psa").

- Objetivo del ejercicio: Comparar las tasas de conversión entre los grupos de prueba "ad" y "psa" para determinar si el nuevo diseño de anuncio es más efectivo en la conversión de usuarios.

- Pasos que debemos seguir:

    1. Preparación de los datos: Asegúrate de que los datos estén limpios y listos para el análisis. Esto incluye la eliminación de datos faltantes y la verificación de la coherencia de los datos.

    2. Definición de las hipótesis: Plantea una hipótesis nula (H0) y una hipótesis alternativa (H1) para el experimento (en este caso os las damos definidas):

        - Hipótesis nula (H0): No hay diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

        - Hipótesis alternativa (H1): Existe una diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

    3. Cálculo de la tasa de conversión: Calcula la tasa de conversión para cada grupo:

        - Para el grupo "ad", la tasa de conversión es el número de usuarios convertidos dividido por el número total de usuarios en ese grupo.

        - Para el grupo "psa", realiza el mismo cálculo.

    4. Prueba de hipótesis: Utiliza una prueba estadística, como la prueba t de Student o la prueba chi-cuadrado, para comparar las tasas de conversión entre los grupos "ad" y "psa".

    5. Análisis de resultados:

        - Si el valor p es menor que un umbral predefinido (por ejemplo, 0.05), rechazamos la hipótesis nula y concluimos que hay una diferencia significativa en las tasas de conversión entre los grupos.

        - Si el valor p es mayor que el umbral, no podemos rechazar la hipótesis nula y concluimos que no hay evidencia suficiente para afirmar que hay una diferencia significativa.

    6. Conclusiones y recomendaciones: Basándote en los resultados de la prueba de hipótesis, llega a una conclusión sobre si el nuevo diseño de anuncio es más efectivo en la conversión de usuarios que el diseño anterior. Si es así, puedes hacer recomendaciones basadas en estos hallazgos.

In [9]:
df_marketing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user id        588101 non-null  int64 
 1   test group     588101 non-null  object
 2   converted      588101 non-null  bool  
 3   total ads      588101 non-null  int64 
 4   most ads day   588101 non-null  object
 5   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 27.5+ MB


In [12]:
df_marketing.duplicated().sum()

0

In [14]:
df_marketing.describe().T

,count,mean,std,min,25%,50%,75%,max
user id,588101.0,1.310692e+06,202225.983128,900000.0,1143190.0,1313725.0,1484088.0,1654483.0
total ads,588101.0,2.482088e+01,43.715181,1.0,4.0,13.0,27.0,2065.0
most ads hour,588101.0,1.446906e+01,4.834634,0.0,11.0,14.0,18.0,23.0


In [15]:
df_marketing.describe(include=object).T

,count,unique,top,freq
test group,588101,2,ad,564577
most ads day,588101,7,Friday,92608


In [16]:
df_marketing.sample(1)

,user id,test group,converted,total ads,most ads day,most ads hour
426333,1633737,ad,False,15,Friday,12


In [22]:
df_marketing['CR'] = df_marketing['converted'].sum() / df_marketing['total ads'].sum()

In [19]:
df_marketing['converted'].sum()

14843

In [21]:
df_marketing['total ads'].sum()

14597182

In [31]:
df_ad = df_marketing[df_marketing['test group']== 'ad']
df_ad

,user id,test group,converted,total ads,most ads day,most ads hour,CR
0,1069124,ad,False,130,Monday,20,0.001017
1,1119715,ad,False,93,Tuesday,22,0.001017
2,1144181,ad,False,21,Tuesday,18,0.001017
3,1435133,ad,False,355,Tuesday,10,0.001017
4,1015700,ad,False,276,Friday,14,0.001017
...,...,...,...,...,...,...,...
588096,1278437,ad,False,1,Tuesday,23,0.001017
588097,1327975,ad,False,1,Tuesday,23,0.001017
588098,1038442,ad,False,3,Tuesday,23,0.001017
588099,1496395,ad,False,1,Tuesday,23,0.001017


In [35]:
df_psa = df_marketing[df_marketing['test group']== 'psa']
df_psa

,user id,test group,converted,total ads,most ads day,most ads hour,CR
18,900681,psa,False,248,Saturday,19,0.001017
38,905704,psa,False,27,Thursday,8,0.001017
68,904595,psa,False,13,Tuesday,19,0.001017
140,901904,psa,False,32,Wednesday,19,0.001017
157,902234,psa,False,105,Tuesday,19,0.001017
...,...,...,...,...,...,...,...
588052,900959,psa,False,16,Tuesday,22,0.001017
588063,902828,psa,False,3,Tuesday,22,0.001017
588066,914578,psa,False,1,Tuesday,22,0.001017
588069,909042,psa,False,6,Tuesday,22,0.001017


In [26]:
df_marketing.sample(5)

,user id,test group,converted,total ads,most ads day,most ads hour,CR
581237,1589500,ad,False,2,Monday,13,0.001017
502141,1230871,ad,False,2,Sunday,11,0.001017
548745,1098632,ad,False,6,Monday,23,0.001017
434610,1065179,ad,False,7,Saturday,12,0.001017
211434,917559,psa,False,1,Wednesday,18,0.001017


In [36]:
tabla_cont = pd.crosstab(df_marketing['converted'], df_marketing['test group'])
tabla_cont

test group,ad,psa
converted,,
False,550154,23104
True,14423,420


In [37]:
# H0= si el valor es menor que 0.05 (los datos son normales)  H1= si es mayor a 0.05

chi2, p_value, _, _ = chi2_contingency(tabla_cont)
print(p_value)

1.9989623063390075e-13


In [32]:
df_ad['converted'].mean()

0.025546559636683747

### Ejercicio 2:

En este caso, vamos a realizar un experimento para comparar la cantidad promedio de anuncios vistos por los usuarios en los grupos "ad" y "psa" y determinar si hay una diferencia significativa entre ellos.

- Objetivo del experimento: Comparar la cantidad promedio de anuncios vistos por usuario entre los grupos de prueba "ad" y "psa" para determinar si el tipo de anuncio afecta la cantidad de anuncios que los usuarios ven.

- Preparación de los datos: Asegúrate de que los datos estén limpios y listos para el análisis. Esto incluye la eliminación de datos faltantes y la verificación de la coherencia de los datos.

- Hipótesis: Plantea una hipótesis nula (H0) y una hipótesis alternativa (H1) para el experimento. En este caso, las deberéis plantear vosotras

- Prueba de hipótesis: Utiliza una prueba estadística, como la prueba t de Student, para comparar la cantidad promedio de anuncios vistos por usuario entre los grupos "ad" y "psa". Esto te permitirá determinar si hay una diferencia significativa entre los dos grupos.

- Análisis de resultados:

    - Si el valor p (p-value) es menor que un umbral predefinido (por ejemplo, 0.05), rechazamos la hipótesis nula y concluimos que hay una diferencia significativa en la cantidad promedio de anuncios vistos por usuario entre los grupos.

    - Si el valor p es mayor que el umbral, no podemos rechazar la hipótesis nula y concluimos que no hay evidencia suficiente para afirmar que hay una diferencia significativa.

- Conclusiones y recomendaciones: Basándote en los resultados de la prueba de hipótesis, llega a una conclusión sobre si el tipo de anuncio (ad o psa) tiene un impacto significativo en la cantidad promedio de anuncios vistos por usuario. Si es así, puedes hacer recomendaciones basadas en estos hallazgos.